In [5]:
import os, yaml, shutil, glob
from ultralytics import YOLO
from roboflow import Roboflow
from IPython.display import Image, display

# --- 1. DOWNLOAD DATASETS ---

rf = Roboflow(api_key="BJDNkOnpT2z6XuaFpI94") # REPLACE WITH YOUR NEW KEY IF YOU REVOKED THE OLD ONE

# Dataset 1: Accessible-Toilets
p1 = rf.workspace("image-bounding-for-datasets").project("accessible-toilets")
v1 = p1.version(1)
ds1 = v1.download("yolov8")

# Dataset 2: Bathroom-SDSNE
p2 = rf.workspace("adrian-gorcea").project("bathroom-sdsne")
v2 = p2.version(2)
ds2 = v2.download("yolov8")

# Dataset 3: Door-handle (FIXED)
p3 = rf.workspace("new-workspace-vjwug").project("door-handle-hzojf")
v3 = p3.version(1)        # Changed from p2 to p3
ds3 = v3.download("yolov8") # Changed from v2 to v3


# --- 2. DEFINE MAPPINGS ---

# Mapping for dataset 1 (Accessible Toilets)
dataset1_map = {
    0: 1,   # grab_handle -> class 1
    1: 0,   # toilet      -> class 0
    2: 6    # wheelchair  -> class 6
}

# Mapping for dataset 2 (Bathroom SDSNE)
dataset2_map = {
    0: 2,    # mirror
    1: 3,    # sink
    2: 4,    # soap
    3: 5,    # towel
    4: None, # DELETE garbage
    5: 0     # wc -> toilet
}

# Mapping for dataset 3 (Door Handle) -> NEW CLASS 7
# Assuming Dataset 3 has only one class "door-handle" at index 0
dataset3_map = {
    0: 7     # door-handle -> class 7 (New Class)
}


# --- 3. REWRITE LABELS ---

def rewrite_labels(folder, id_map):
    if not os.path.exists(folder):
        print(f"Skipping {folder} (not found)")
        return
    
    print(f"Processing labels in: {folder}")
    for f in os.listdir(folder):
        if not f.endswith(".txt"):
            continue

        old_path = os.path.join(folder, f)
        new_lines = []

        with open(old_path) as fp:
            for line in fp:
                parts = line.strip().split()
                old_id = int(parts[0])

                if old_id not in id_map:
                    continue
                new_id = id_map[old_id]
                if new_id is None:
                    continue

                new_lines.append(" ".join([str(new_id)] + parts[1:]))

        with open(old_path, "w") as fp:
            fp.write("\n".join(new_lines))

# Apply to Dataset 1
for split in ["train", "test", "valid"]:
    rewrite_labels(f"{ds1.location}/{split}/labels", dataset1_map)

# Apply to Dataset 2
for split in ["train", "test", "valid"]:
    rewrite_labels(f"{ds2.location}/{split}/labels", dataset2_map)

# Apply to Dataset 3 (New)
for split in ["train", "test", "valid"]:
    rewrite_labels(f"{ds3.location}/{split}/labels", dataset3_map)


# --- 4. MERGE FOLDERS ---

# Create combined structure
for t in ["train", "valid", "test"]:
    os.makedirs(f"combined/images/{t}", exist_ok=True)
    os.makedirs(f"combined/labels/{t}", exist_ok=True)

def merge(src, split):
    img_s = f"{src}/{split}/images"
    lbl_s = f"{src}/{split}/labels"
    img_d = f"combined/images/{split}"
    lbl_d = f"combined/labels/{split}"
    
    if os.path.exists(img_s):
        shutil.copytree(img_s, img_d, dirs_exist_ok=True)
    if os.path.exists(lbl_s):
        shutil.copytree(lbl_s, lbl_d, dirs_exist_ok=True)

# Merge all 3 datasets
# We use ds.location to ensure we get the exact folder name Roboflow downloaded
for split in ["train", "test", "valid"]:
    merge(ds1.location, split)
    merge(ds2.location, split)
    merge(ds3.location, split)


# --- 5. CREATE YAML ---

combined_yaml = {
    "train": "combined/images/train",
    "val": "combined/images/valid",
    "test": "combined/images/test",
    "names": [
        "toilet",          # 0
        "grab_handle",     # 1
        "mirror",          # 2
        "sink",            # 3
        "soap",            # 4
        "towel",           # 5
        "wheelchair_logo", # 6
        "door-handle"      # 7 (NEW FROM DATASET 3)
    ],
    "nc": 8 # Total classes
}

with open("combined.yaml", "w") as f:
    yaml.dump(combined_yaml, f)


# --- 6. TRAIN ---

model = YOLO("yolov8n.pt")

model.train(
    data="combined.yaml",
    epochs=100,
    imgsz=640,
    batch=8
)

# --- 7. INFERENCE ---

from ipywidgets import FileUpload

# Reload best model
model = YOLO("runs/detect/train/weights/best.pt")
print("Corrected Classes:", model.model.names)

# Create uploader
uploader = FileUpload(accept='image/*', multiple=False)
display(uploader)

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Door-handle-1 in yolov8:: 100%|██████████| 1148/1148 [00:00<00:00, 1905.06it/s]


Processing labels in: C:\Users\Ryan.H\PycharmProjects\Toilet-Accessibility-Detection\Accessible-Toilets-1/train/labels
Processing labels in: C:\Users\Ryan.H\PycharmProjects\Toilet-Accessibility-Detection\Accessible-Toilets-1/test/labels
Skipping C:\Users\Ryan.H\PycharmProjects\Toilet-Accessibility-Detection\Accessible-Toilets-1/valid/labels (not found)
Processing labels in: C:\Users\Ryan.H\PycharmProjects\Toilet-Accessibility-Detection\bathroom-2/train/labels
Processing labels in: C:\Users\Ryan.H\PycharmProjects\Toilet-Accessibility-Detection\bathroom-2/test/labels
Processing labels in: C:\Users\Ryan.H\PycharmProjects\Toilet-Accessibility-Detection\bathroom-2/valid/labels
Processing labels in: C:\Users\Ryan.H\PycharmProjects\Toilet-Accessibility-Detection\Door-handle-1/train/labels
Processing labels in: C:\Users\Ryan.H\PycharmProjects\Toilet-Accessibility-Detection\Door-handle-1/test/labels
Processing labels in: C:\Users\Ryan.H\PycharmProjects\Toilet-Accessibility-Detection\Door-handle

KeyboardInterrupt: 